In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import tqdm
import optuna
%matplotlib inline

from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device} will be used for training...")

cuda will be used for training...


In [4]:
train_data_path = os.path.join("..", "dataset", "prototyping", "train")
valid_data_path = os.path.join("..", "dataset", "prototyping", "valid")
print(f"Train path: {train_data_path}\nValid path: {valid_data_path}")

Train path: ..\dataset\prototyping\train
Valid path: ..\dataset\prototyping\valid


# 1. Defining Transforms for Data Augumentation

In [5]:
image_size = 256

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize([0, 0, 0], [1, 1, 1]),
])

test_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize([0, 0, 0], [1, 1, 1]),
])

In [6]:
training_data = ImageFolder(train_data_path, transform=train_transforms)
valid_data = ImageFolder(valid_data_path, transform=test_transforms)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True)

In [7]:
idx_to_class = {v: k for k, v in training_data.class_to_idx.items()}
idx_to_class

{0: 'fake', 1: 'real'}

In [8]:
def display_image(img_tensor, title=None):
  image = img_tensor.to("cpu").clone().detach()
  image = image.numpy().squeeze()
  image = image.transpose(1, 2, 0)
  image = image
  # image = image.clip(0, 1)

  if title:
    plt.title(title)

  plt.imshow(image)
  plt.show()

In [9]:
images, labels = next(iter(train_loader))
display_image(images[0], idx_to_class[labels[0].item()])

In [10]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights)

d:\Programming\Hackathons\SPS Cup\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
for param in model.parameters():
    param.requires_grad = False

In [17]:
def objective(trial: optuna.Trial):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.4)
    layers = []

    in_features = 512
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 128, 512)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = out_features
    
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LogSoftmax(dim=1))
    fc = nn.Sequential(*layers)
    model.fc = fc

    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    epochs = 6

    model.train()
    model.to(device)
    for epoch in range(epochs):
        for images, labels in tqdm.tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy

In [18]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2024-12-23 00:49:13,356] A new study created in memory with name: no-name-69da06ab-9f74-40a9-ae67-f164a36e25a6
C:\Users\navan\AppData\Local\Temp\ipykernel_9612\2021838525.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
C:\Users\navan\AppData\Local\Temp\ipykernel_9612\2021838525.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.4)
100%|██████████| 375/375 [00:25<00:00, 14.78it/s]
[I 2024-12-23 00:52:10,509] Trial 0 finished with value: 0.67 and parameters: {'n_layers': 1, 'learning_rate': 0.00017168186205086537, 'dropout_rate': 0.11229880051397158, 'n_

In [381]:
# [I 2024-12-23 01:29:01,081] Trial 14 finished with value: 0.735 and parameters: {'n_layers': 2, 'learning_rate': 0.002571768221420213, 'dropout_rate': 0.18647655711328523, 'n_units_l0': 1216, 'n_units_l1': 2005}. Best is trial 14 with value: 0.735.

classifier = torch.nn.Sequential(
    nn.Linear(2048, 2),
    nn.LogSoftmax(dim=1),
)
classifier

Sequential(
  (0): Linear(in_features=2048, out_features=2, bias=True)
  (1): LogSoftmax(dim=1)
)

In [382]:
for param in model.parameters():
    param.requires_grad = False

model.fc = classifier
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [383]:
checkpoint_path = "checkpoint.pt"

# Hyper parameters

In [384]:
criterion = nn.NLLLoss()
epochs = 50
learning_rate = 0.005
validate_every = 5

In [385]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [386]:
validation_loss_min = np.Inf
model.to(device)

for e in range(epochs):
  running_loss = 0

  print(f"EPOCH {e + 1}:")
  i = 0
  for images, labels in tqdm.tqdm(train_loader):
    i += 1
    # print(f"{i} / {len(train_loader)} image being used for training")
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model.forward(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {e + 1} :: Running Loss: {running_loss / len(train_loader)}")

  validation_loss = 0

  if (e + 1) % validate_every == 0:
    with torch.no_grad():
      model.eval()

      for images, labels in tqdm.tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)

        logps = model.forward(images)
        loss = criterion(logps, labels)

        validation_loss += loss.item()

    model.train()

    running_loss /= len(train_loader)
    validation_loss /= len(valid_loader)

    print(f"EPOCH {e + 1}:")
    print(f"Running Loss = {running_loss}\tValidation Loss: {validation_loss}")

    if validation_loss <= validation_loss_min:
      torch.save({"state_dict": model.state_dict(), "valid_loss": validation_loss_min}, checkpoint_path)
      print(f"Minimum Validtion loss acquired, {validation_loss_min} to {validation_loss}")
      validation_loss_min = validation_loss


EPOCH 1:


100%|██████████| 375/375 [02:12<00:00,  2.84it/s]


Epoch 1 :: Running Loss: 0.6560902999242146
EPOCH 2:


100%|██████████| 375/375 [02:11<00:00,  2.84it/s]


Epoch 2 :: Running Loss: 0.6064608128070831
EPOCH 3:


...

Epoch 10 :: Running Loss: 0.5574892037709555


100%|██████████| 7/7 [00:02<00:00,  3.26it/s]


EPOCH 10:
Running Loss = 0.5574892037709555	Validation Loss: 1.0516889095306396
EPOCH 11:


  8%|▊         | 31/375 [00:11<02:05,  2.73it/s]


KeyboardInterrupt: 

# Metrics Calculation

In [1]:
validation_loss = 0
correct_valid = 0
total_valid = 0

with torch.no_grad():
    model.eval()

    for images, labels in tqdm.tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)

        logps = model.forward(images)
        loss = criterion(logps, labels)
        validation_loss += loss.item()

        # Calculate validation accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        correct_valid += (top_class == labels.view(*top_class.shape)).sum().item()
        total_valid += labels.size(0)

validation_loss /= len(valid_loader)
validation_accuracy = correct_valid / total_valid * 100

print("Validation Accuracy:", validation_accuracy)

NameError: name 'torch' is not defined